In [2]:
%matplotlib ipympl
from QDP import qdp
import os.path
import numpy as np
from scipy.stats import poisson
import matplotlib.pyplot as plt
from scipy import optimize
dp = '/home/ebert/jupyter-notebooks/FNode-data'
out = '/home/ebert/jupyter-notebooks/FNode/analyses/CollisionPaper/trap-freqs/'
exclude = [2]

def gaussian(x, a, x0, s, b):
    return b + a*np.exp(-0.5*((x-x0)/s)**2)

single_only = False

ret = 'retention'
err = 'error'
if single_only:
    ret = 'conditional_retention'
    err = 'conditional_retention_err'

In [3]:
exp_name = '2018_07_07_19_58_34_p-heating-radial-freq-scan'
if exp_name:
    exp_date = '_'.join(exp_name.split('_')[:3])
    experiment_file = os.path.join(exp_date, exp_name, 'results.hdf5')
else:
    experiment_file = ''
q = qdp.QDP(base_data_path=dp)
q.load_data_file(experiment_file)
print "ivars: {}".format(q.experiments[0]['variable_list'])
cs_cuts = [[23],[3]]
rb_cuts = [[23],[3]]
q.set_thresholds(cs_cuts, roi=0)
q.set_thresholds(rb_cuts, roi=1)
q.generate_thresholds(loss=False)
retention = q.apply_thresholds(exclude_rois=exclude, ncondition=1)
cuts = q.get_thresholds()
roi_map = ['cs', 'rb']
fmt_map = ['bo', 'rd']
marker_styles = [
    dict(color='cornflowerblue', marker='o', markeredgecolor='b', linestyle='None'),
    dict(color='indianred', marker='d', markeredgecolor='r', linestyle='None')
]
entries = np.product(retention[ret].shape[:-1])
tbl_str = "ivar: {0:.4f}\tcloading: {1:.3f}\trloading: {2:.3f}\t\tcretention: {3:.3f} +- {4:.3f}\trretention: {5:.3f} +- {6:.3f}"
for i in range(entries):
    print(tbl_str.format(
        retention['ivar'].reshape((entries,-1))[i][0],
        retention['loading'].reshape((entries,-1))[i][0], retention['loading'].reshape((entries,-1))[i][1],
        retention[ret].reshape((entries,-1))[i][0], retention[err].reshape((entries,-1))[i][0],
        retention[ret].reshape((entries,-1))[i][1], retention[err].reshape((entries,-1))[i][1]
    ))
q.save_experiment_data()
xlab = q.experiments[0]['variable_desc'][q.experiments[0]['variable_list'][0]]['description']

# plotting
fig, ax = plt.subplots(nrows=2, figsize=(5,8))
plt.subplots_adjust(wspace=0.3)
popts = []
pcovs = []
for i, yvals in enumerate([ret, 'loading']):
    for e in range(len(retention['ivar'])):
        for r in range(2):            
            if yvals==ret:
                try:
                    ax[i].errorbar(retention['ivar'][e, :, r], retention[yvals][e, :, r],
                        yerr=retention[err][e,:,r],
                        **marker_styles[r]
                    )
                except Exception as ee:
                    print(ee)
                    ax[i].plot(retention['ivar'][e, :, 0], retention[yvals][e, :, r], **marker_styles[r])
                # fitting
                xs = np.linspace(min(retention['ivar'][e, :, 0]), max(retention['ivar'][e, :, 0]),100)
                guess = [-0.4, retention['ivar'][e, np.argmin(retention[yvals][e, :, r]), 0], 5, 0.8]
                try:
                    popt, pcov = optimize.curve_fit(
                        gaussian,
                        retention['ivar'][e, :, 0],
                        retention[yvals][e, :, r],
                        p0=guess,
                        sigma=retention[err][e, :, r]
                    )
                    popts.append(popt)
                    pcovs.append(pcov)
                except Exception as ee:
                    print ee
                else:
                    ax[i].plot(xs, gaussian(xs, *popt), fmt_map[r][0]+'--', alpha=0.6)
                    print(popt)
                    fit_data = "{}\na: {:.3f}\nx0: {:.3f}\ns: {:.3f}\nb: {:.3f}"
                    start = np.min(retention['ivar'][e, :, 0])
                    stop = np.max(retention['ivar'][e, :, 0])
                    dx = (stop-start)/10
                    ax[i].text(start+dx+r*5*dx, 0.1, fit_data.format(roi_map[r],*popt), fontsize=12)
            else:
                ax[i].plot(retention['ivar'][e, :, 0], retention[yvals][e, :, r], **marker_styles[r])
                
        
    ax[i].set_ylim(0., 1)
    ax[i].set_xlabel(xlab)
    ax[i].set_ylabel(yvals)

fmts = ['pdf', 'png', 'eps']
fn = os.path.join(out, 'radial_fit.{}')
for fmt in fmts:
    fig.savefig(fn.format(fmt), dpi=200, format=fmt)

Command '['git', 'describe', '--always']' returned non-zero exit status 128
data at: 2018_07_07/2018_07_07_19_58_34_p-heating-radial-freq-scan/results.hdf5
name 'fort_exp' is not defined
name 'rb_uwave_freq' is not defined
ivars: [u'p_heating_freq']


/home/ebert/jupyter-notebooks/FNode/analyses/qdp/QDP/qdp.py:540: VisibleDeprecationWarning: Passing `normed=True` on non-uniform bins has always been broken, and computes neither the probability density function nor the probability mass function. The result is only correct if the bins are uniform, when density=True will produce the same result anyway. The argument will be removed in a future version of numpy.
  hist, bin_edges = np.histogram(shot_data, bins=hbins, normed=True)


--------------------
--------------------
--------------------
--------------------
Internal work array size computation failed: -5
There may be some issue with your guess: `[7.38507555e-18 0.00000000e+00 0.00000000e+00]`
--------------------
Internal work array size computation failed: -5
There may be some issue with your guess: `[7.38507555e-18 0.00000000e+00 0.00000000e+00]`
--------------------
ivar: 60.0000	cloading: 0.289	rloading: 0.108		cretention: 0.908 +- 0.030	rretention: 0.856 +- 0.030
ivar: 61.3889	cloading: 0.260	rloading: 0.089		cretention: 0.838 +- 0.031	rretention: 0.832 +- 0.031
ivar: 62.7778	cloading: 0.279	rloading: 0.152		cretention: 0.879 +- 0.027	rretention: 0.889 +- 0.027
ivar: 64.1667	cloading: 0.251	rloading: 0.174		cretention: 0.769 +- 0.035	rretention: 0.788 +- 0.035
ivar: 65.5556	cloading: 0.232	rloading: 0.205		cretention: 0.885 +- 0.035	rretention: 0.780 +- 0.035
ivar: 66.9444	cloading: 0.262	rloading: 0.163		cretention: 0.843 +- 0.038	rretention: 0.727 +

/home/ebert/jupyter-notebooks/FNode/analyses/qdp/QDP/qdp.py:131: RuntimeWarning: overflow encountered in double_scalars
  errs[r] = (z/n[r].astype('float'))*np.sqrt(ns[r].astype('float')*(1.0-ns[r].astype('float')/n[r].astype('float')))
/home/ebert/jupyter-notebooks/FNode/analyses/qdp/QDP/qdp.py:131: RuntimeWarning: invalid value encountered in sqrt
  errs[r] = (z/n[r].astype('float'))*np.sqrt(ns[r].astype('float')*(1.0-ns[r].astype('float')/n[r].astype('float')))


FigureCanvasNbAgg()

[-0.32582662 74.43407415  2.84930846  0.88724338]
[-0.4748903  70.91853927  3.35641371  0.91164276]


In [4]:
q.experiments[0]['iterations'][0]['variables']['fort_drop_us']

6

In [5]:
qax.experiments[0]['iterations'][0]['variables']['p_heating_duration']

NameError: name 'qax' is not defined

In [ ]:
for k in q.experiments[0]['iterations'][0]['variables'].keys():
    if 'heat' in k:
        print k

In [6]:
exp_name = '2018_07_07_18_40_05_p-heating-axial-freq-scan'
if exp_name:
    exp_date = '_'.join(exp_name.split('_')[:3])
    experiment_file = os.path.join(exp_date, exp_name, 'results.hdf5')
else:
    experiment_file = ''
qax = qdp.QDP(base_data_path=dp)
qax.load_data_file(experiment_file)
print "ivars: {}".format(q.experiments[0]['variable_list'])
cs_cuts = [[23],[3]]
rb_cuts = [[23],[3]]
qax.set_thresholds(cs_cuts, roi=0)
qax.set_thresholds(rb_cuts, roi=1)
qax.generate_thresholds(loss=False)
retention_ax = qax.apply_thresholds(exclude_rois=exclude, ncondition=1)
cuts = qax.get_thresholds()
entries = np.product(retention_ax[ret].shape[:-1])
tbl_str = "ivar: {0:.4f}\tcloading: {1:.3f}\trloading: {2:.3f}\t\tcretention: {3:.3f} +- {4:.3f}\trretention: {5:.3f} +- {6:.3f}"
for i in range(entries):
    print(tbl_str.format(
        retention_ax['ivar'].reshape((entries,-1))[i][0],
        retention_ax['loading'].reshape((entries,-1))[i][0], retention_ax['loading'].reshape((entries,-1))[i][1],
        retention_ax[ret].reshape((entries,-1))[i][0], retention_ax[err].reshape((entries,-1))[i][0],
        retention_ax[ret].reshape((entries,-1))[i][1], retention_ax[err].reshape((entries,-1))[i][1]
    ))
qax.save_experiment_data()
xlab = qax.experiments[0]['variable_desc'][qax.experiments[0]['variable_list'][0]]['description']

# plotting
fig, ax = plt.subplots(nrows=2, figsize=(5,8))
plt.subplots_adjust(wspace=0.3)
popts_ax = []
pcovs_ax = []
for i, yvals in enumerate([ret, 'loading']):
    for e in range(len(retention_ax['ivar'])):
        for r in range(2):            
            if yvals==ret:
                try:
                    ax[i].errorbar(retention_ax['ivar'][e, :, r], retention_ax[yvals][e, :, r],
                        yerr=retention_ax[err][e,:,r],
                        **marker_styles[r]
                    )
                except Exception as ee:
                    print(ee)
                    ax[i].plot(retention_ax['ivar'][e, :, 0], retention_ax[yvals][e, :, r], **marker_styles[r])
                # fitting
                xs = np.linspace(min(retention_ax['ivar'][e, :, 0]), max(retention_ax['ivar'][e, :, 0]),100)
                guess = [-0.4, retention_ax['ivar'][e, np.argmin(retention_ax[yvals][e, :, r]), 0], 0.21, 0.8]
                try:
                    popt_ax, pcov_ax = optimize.curve_fit(
                        gaussian,
                        retention_ax['ivar'][e, :, 0],
                        retention_ax[yvals][e, :, r],
                        p0=guess,
                        sigma=retention_ax[err][e, :, r]
                    )
                    popts_ax.append(popt_ax)
                    pcovs_ax.append(pcov_ax)
                except Exception as ee:
                    print ee
                else:
                    ax[i].plot(xs, gaussian(xs, *popt_ax), fmt_map[r][0]+'--', alpha=0.6)
                    print(popt_ax)
                    fit_data = "{}\na: {:.3f}\nx0: {:.3f}\ns: {:.3f}\nb: {:.3f}"
                    start = np.min(retention_ax['ivar'][e, :, 0])
                    stop = np.max(retention_ax['ivar'][e, :, 0])
                    dx = (stop-start)/10
                    ax[i].text(start+dx+r*5*dx, 0.1, fit_data.format(roi_map[r],*popt_ax), fontsize=12)
            else:
                ax[i].plot(retention_ax['ivar'][e, :, 0], retention_ax[yvals][e, :, r], **marker_styles[r])
                
        
    ax[i].set_ylim(0., 1)
    ax[i].set_xlabel(xlab)
    ax[i].set_ylabel(yvals)

fmts = ['pdf', 'png', 'eps']
fn = os.path.join(out, 'axial_fit.{}')
for fmt in fmts:
    fig.savefig(fn.format(fmt), dpi=200, format=fmt)

Command '['git', 'describe', '--always']' returned non-zero exit status 128
data at: 2018_07_07/2018_07_07_18_40_05_p-heating-axial-freq-scan/results.hdf5
name 'fort_exp' is not defined
name 'rb_uwave_freq' is not defined
ivars: [u'p_heating_freq']
--------------------
--------------------
--------------------
--------------------
Internal work array size computation failed: -5
There may be some issue with your guess: `[8.79965409e-18 0.00000000e+00 0.00000000e+00]`
--------------------
Internal work array size computation failed: -5
There may be some issue with your guess: `[8.79965409e-18 0.00000000e+00 0.00000000e+00]`
--------------------
ivar: 5.5000	cloading: 0.202	rloading: 0.131		cretention: 0.778 +- 0.029	rretention: 0.869 +- 0.029
ivar: 5.6500	cloading: 0.220	rloading: 0.127		cretention: 0.789 +- 0.031	rretention: 0.842 +- 0.031
ivar: 5.8000	cloading: 0.137	rloading: 0.173		cretention: 0.718 +- 0.028	rretention: 0.879 +- 0.028
ivar: 5.9500	cloading: 0.186	rloading: 0.171		cre

FigureCanvasNbAgg()

[-0.1577654   7.47716644  0.2278128   0.78293377]
[-0.31411111  7.13514443  0.22505478  0.8813191 ]


In [7]:
fig, ax = plt.subplots(ncols=2, sharey=True, figsize=(2*3.375,3))
marker_styles = [
    dict(color='cornflowerblue', marker='o', markeredgecolor='b', linestyle='None', markersize=5),
    dict(color='indianred', marker='d', markeredgecolor='r', linestyle='None', markersize=5)
]

i=0
ax[i].set_ylim(0,1)
ax[i].set_xlim(5.4,8.6)
ax[i].set_xlabel(r'$\Omega/2\pi$ (kHz)', fontsize=14)
ax[i].set_ylabel('Retention', fontsize=14)
ax[i].text(8.2, 0.05, 'a)', fontsize=14)
ax[i].grid()
for r in range(2):
    ax[i].errorbar(retention_ax['ivar'][e, :, r], retention_ax[ret][e, :, r],
        yerr=retention_ax[err][e,:,r],
        **marker_styles[r]
    )
    # fitting
    xs = np.linspace(min(retention_ax['ivar'][e, :, 0])-1, max(retention_ax['ivar'][e, :, 0])+1,100)
    ax[i].plot(xs, gaussian(xs, *popts_ax[r]), fmt_map[r][0]+'--', alpha=0.6)
    print(popts_ax[r][1])
    print(np.sqrt(np.abs(np.diag(pcovs_ax[r])))[1])

i=1
ax[i].set_ylim(0,1)
ax[i].set_xlim(59,86)
ax[i].set_xlabel(r'$\Omega/2\pi$ (kHz)', fontsize=14)
ax[i].text(82, 0.05, 'b)', fontsize=14)
ax[i].grid(which='both')
for r in range(2):
    ax[i].errorbar(retention['ivar'][e, :, r], retention[ret][e, :, r],
        yerr=retention[err][e,:,r],
        **marker_styles[r]
    )
    # fitting
    xs = np.linspace(min(retention['ivar'][e, :, 0])-1, max(retention['ivar'][e, :, 0])+1,100)
    ax[i].plot(xs, gaussian(xs, *popts[r]), fmt_map[r][0]+'--', alpha=0.6)
    print(popts[r][1])
    print(np.sqrt(np.abs(np.diag(pcovs[r])))[1])
    
plt.tight_layout()
fmts = ['pdf', 'png', 'eps']
fn = os.path.join(out, 'dual_fit.{}')
for fmt in fmts:
    fig.savefig(fn.format(fmt), dpi=200, format=fmt)

FigureCanvasNbAgg()

7.477166444703718
0.056291597554227975
7.135144429627263
0.03083129023743823
74.43407414548292
0.3680805844227694
70.91853926632811
0.22198689930038162


In [8]:
qax.experiments[0]['iterations'][0]['variables']['fort_drop_us']

6

In [29]:
l = 1.064  # odt wavelength
wr = []
swr = []
wa = []
swa = []
w0 = []
sw0 = []
for r in range(2):
    wr.append(popts[r][1]/2.)
    swr.append(np.sqrt(np.abs(np.diag(pcovs[r])))[1])
    wa.append(popts_ax[r][1]/2.)
    swa.append(np.sqrt(np.abs(np.diag(pcovs_ax[r])))[1])
    
    w0.append(l*wr[-1]/wa[-1]/(np.sqrt(2)*np.pi))
    sw0.append(l/(np.sqrt(2)*np.pi)*np.sqrt((swr[-1]/wa[-1])**2 + (wr[-1]*swa[-1]/wa[-1]**2)**2))

print("Est. ODT waist")
print(w0)
print(sw0)
w0avg = np.dot(np.power(sw0,-2), w0)/np.sum(np.power(sw0,-2))
print(w0avg)
print(np.sqrt(np.sum(np.power(sw0,2)))/2)
# print(wr,swr,wa,swa)

print("\n(wr_cs/wr_rb)^2")
print((wr[0]/wr[1])**2)
print("\n(wa_cs/wa_rb)^2")
print((wa[0]/wa[1])**2)

Est. ODT waist
[2.3840288125901794, 2.3803113673026504]
[0.04294730523150262, 0.025401118895268488]
2.3812747644965473
0.024948386058918814

(wr_cs/wr_rb)^2
1.10160023030796

(wa_cs/wa_rb)^2
1.0981674313724374


In [10]:
popts_ax

[array([-0.1577654 ,  7.47716644,  0.2278128 ,  0.78293377]),
 array([-0.31411111,  7.13514443,  0.22505478,  0.8813191 ])]

In [11]:
pcovs

[array([[ 1.51527740e-03,  1.66298305e-04,  5.13922597e-03,
         -1.29642179e-04],
        [ 1.66298305e-04,  1.35483317e-01, -3.76239563e-02,
         -9.24681162e-05],
        [ 5.13922597e-03, -3.76239563e-02,  1.55805240e-01,
          3.24213166e-03],
        [-1.29642179e-04, -9.24681162e-05,  3.24213166e-03,
          2.26652420e-04]]),
 array([[ 9.03765766e-04, -1.34958382e-04,  2.47604389e-03,
         -7.73704741e-05],
        [-1.34958382e-04,  4.92781835e-02,  3.81949578e-04,
          6.92551050e-05],
        [ 2.47604389e-03,  3.81949578e-04,  6.31800725e-02,
          1.78331095e-03],
        [-7.73704741e-05,  6.92551050e-05,  1.78331095e-03,
          1.45211234e-04]])]

In [14]:
(37.21703707274146/35.459269633164055)**2

1.10160023030796

In [ ]:
(3.738583222351859/